### this script has been taken from the checkFiltering and DivideTrialsIntoReadingAndWriting script, and so all the filtering is based on the same

In [1]:
%matplotlib qt
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
#import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from itertools import *
from operator import *

from openpyxl import load_workbook

In [2]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    '2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [3]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [4]:
TimeDwellOrig = 800
TimeFixation = 300

In [5]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [6]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [7]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dic_dwellTimeOrig_not800:
        TimeDwellOrig = dic_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [8]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [9]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    

    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
            
    return trials    

In [10]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [11]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [12]:
def FindExptStartEndTimes(KeysSelected, timeTyping, full_path):
    # function to find start and end of tasks in experiments
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    timeTrialDict = dict()
    timeTrialDict = {'start': [],
                    'end':[]}
    
    nTrial = -1
    
    
    
    for keys in KeysSelected:
        
            
        
        if keys[1] == 'NextPhrase':
            nTrial = nTrial + 1
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            if nTrial != 0:
                # print('end: ', endTimeTrial)
                #print('')
                timeTrialDict['end'].append(endTimeTrial)
            
            
            startTimeTrial = endTimeTrial + datetime.timedelta(seconds=5)
            
            #print('start: ', startTimeTrial)
            timeTrialDict['start'].append(startTimeTrial)
        
    del timeTrialDict['start'][-1]
    
    
    # remove the extra selections of NewPhrase at the end of some sessions
    replacingList = []
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionOfNextPhrase, timeTrialDict, replacingList)
    
    timeTrialDict_copy = copy.deepcopy(timeTrialDict)
    
    # separate the reading and writing trials for some participants who read in the actual trial, but write in the next
    # trial
    if session_folder_name in dict_keySelection_ReadingTrials:
        # check the reading and writing separate dictionaries
        print('reading and writing sessions are separate')
        
        #print(len(timeTrialDict['start']))
        # writing trials - 
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_WritingTrials, timeTrialDict, replacingList)
        #print(len(timeTrialDict_copy['start']))
        
        # reading trials
        timeTrialDict_reading = findAndRemoveTrials(session_folder_name, dict_keySelection_ReadingTrials, timeTrialDict_copy, replacingList)
    else:
        # some participants skip some sentences, and then it affects the scoreQuestions too. Remove the skipped sentences or 
        # remove the score questions 
        # for these participants, reading and writing trials are the same
        
        #print('same reading and writing trials')
        scoreQuestions = [0, 1, 3, 5, 7, 9, 11]
        timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionNotCompleted, timeTrialDict, scoreQuestions)
        
        # most of the skipped sentences are removed, but for those that are not removed
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_phraseStim, timeTrialDict, replacingList)
        
        timeTrialDict_reading = timeTrialDict_writing 
        
    
          
    return timeTrialDict_reading, timeTrialDict_writing 

In [13]:
def FindReadingPartsOfTrial_inKeysSelected(EventTrials_reading, KeysSelected_new, full_path):
    # find the reading end in the pupil size data
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventReading = dict()    
    EventReading['start'] = list()
    EventReading['end'] = list()
    
    EventReading_index = dict()    
    EventReading_index['start'] = list()
    EventReading_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_reading['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        EventReading['start'].append(startTrialTime)
        EventReading_index['start'].append(startTrialInd)
        
        endTrialTime = EventTrials_reading['end'][ind]
        endTrialInd = KeysSelected_time.index(endTrialTime)
        
        keysSelected_trial = KeysSelected_keys[startTrialInd:endTrialInd]
        
        # compute the end of reading time -- which is taken when the keyboard with phrases key is pressed
        if session_folder_name not in dict_keyboardNotChange_ReadingTrials:
            
            endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
            endReading_keyTime = KeysSelected_time[endReading_keyInd]
            
        else:
            if ind == dict_keyboardNotChange_ReadingTrials[session_folder_name]:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('Sleep') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
            else:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
                
        EventReading['end'].append(KeysSelected_time[endReading_keyInd+1])
        EventReading_index['end'].append(endReading_keyInd+1)
        
        #print(ind)
        #print('reading: ', EventReading['start'][ind], EventReading['end'][ind])
        
    
    
    return EventReading, EventReading_index

In [14]:
def FindWritingPartsOfTrial_inKeysSelected(EventTrials_writing, KeysSelected_new, EventReading):
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventWriting = dict()    
    EventWriting['start'] = list()
    EventWriting['end'] = list()
    
    EventWriting_index = dict()    
    EventWriting_index['start'] = list()
    EventWriting_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_writing['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        endTrialTime = EventTrials_writing['end'][ind]
        endTimeReading = EventReading['end'][ind]
        
        
        # for some participants, reading and writing trials are different. So their writing times will not be the end of 
        # the reading time.
        # Regardless, the writing time should start later than when the reading time ends.
        # We choose the starting time for writing as the one that is later than the start time from writing trials
        # and end time from reading trials
        
        if startTrialTime > endTimeReading:
            EventWriting['start'].append(startTrialTime)
            EventWriting_index['start'].append(startTrialInd)
        else:
            EventWriting['start'].append(endTimeReading)
            EventWriting_index['start'].append(KeysSelected_time.index(endTimeReading))
        
        EventWriting['end'].append(endTrialTime)
        EventWriting_index['end'].append(KeysSelected_time.index(endTrialTime))
        
        #print(ind)
        #print('writing: ', EventWriting['start'][ind], EventWriting['end'][ind])
        
    
    
    return EventWriting, EventWriting_index     

In [15]:
def gazeConvert2ColumnsTo1(GazeLog, columnIndwValidity_list, indValidity):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    #columnInd_list = [joinColumn1_1, joinColumn1_2, joinColumn2_1, joinColumn2_2]
    
    # number of columns in the final dictionary
    nColumns = int(len(columnIndwValidity_list)/2)
    
    # dictionary of columns that are to be joined later
    columns_beforeDecimal = dict()
    columns_afterDecimal = dict()
    
    # dictionary of joined columns
    columnsFinal = dict()
    
    # dictionary to find and equalize missing values in every column
    missingVal_column = dict()
    missingVal = list()
    
    # find correct index of validity column to be used, to find the actual columns relative to that
    columnsValidity_inUse = list()
    
    for ind, row in enumerate(GazeLog):
        #print(ind)
        #print(sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])

        columnsValidity = (sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])
        columnsValidity_inUse.append(int(columnsValidity))
    
    columnsValidity_inUse = np.array(columnsValidity_inUse)
    
    columnInd_list = [[columnsValidity_inUse+i] for i in columnIndwValidity_list]
    
    
    for ind in range(0, nColumns):
        
        dict_name = 'column' + str(ind+1)
        columnsFinal[dict_name] = list()
        columns_afterDecimal[dict_name] = list()
                
        #for indItem, item4 in enumerate(GazeLog):
        #    if 'Invalid' not in item4:
        #        if columnInd_list[2*ind+1][0][indItem] < len(item4):
        #            columns_afterDecimal[dict_name].append(item4[columnInd_list[2*ind+1][0][indItem]])
        #        else:
        #            columns_afterDecimal[dict_name].append('0')
        #    else:
        #        columns_afterDecimal[dict_name].append('nan')
        
                
        columns_beforeDecimal[dict_name] = [item4[columnInd_list[2*ind][0][indItem]] if 'Invalid' not in item4 else 'nan' for indItem, item4 in enumerate(GazeLog)]
        columns_afterDecimal[dict_name] = [item4[columnInd_list[2*ind+1][0][indItem]] if 'Invalid' not in item4 and columnInd_list[2*ind+1][0][indItem] < len(item4) else 'nan' for indItem, item4 in enumerate(GazeLog)]

        
        for i in range(0, len(columns_beforeDecimal[dict_name])):
            if 'Valid' not in columns_beforeDecimal[dict_name][i] and 'Valid' not in columns_afterDecimal[dict_name][i]:
                if 'nan' not in columns_beforeDecimal[dict_name][i] and 'nan' not in columns_afterDecimal[dict_name][i]:
                    if float(columns_afterDecimal[dict_name][i]) > 0: 
                        columnsFinal[dict_name].append(float(columns_beforeDecimal[dict_name][i]+'.'+columns_afterDecimal[dict_name][i]))
                    else:
                        columnsFinal[dict_name].append(np.nan)
                else:
                    columnsFinal[dict_name].append(np.nan)
            else:
                # Rarely, the pupil size is a whole number
                columnsFinal[dict_name].append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
                # right or left eye has whole number pupil size
    
        missingVal_column[dict_name] = np.argwhere(np.isnan(columnsFinal[dict_name]))
        missingVal_column[dict_name] = list(itertools.chain.from_iterable(missingVal_column[dict_name])) # flatten the list
        
        missingVal.extend(missingVal_column[dict_name])
        
        
    
    missingVal = sorted(set(missingVal))
    
    # if one of the columns are nan, the other one is converted too
    for column in range(0, nColumns):
        dict_name = 'column' + str(column+1)
        for ind in missingVal:
            if ind < len(columnsFinal[dict_name]):
                columnsFinal[dict_name][ind] = np.nan
                
    
    return columnsFinal

In [16]:
def hampel(dvec, radius=5, nsigma=3, rem_nomed=False):

    # replace outliers with median values (hampel filter)
    
    mvec = pd.Series(dvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    svec = 1.4862 * np.abs(dvec-mvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    plonk = np.abs(dvec-mvec) > nsigma*svec
    dvec = np.array(dvec)
    dvec[plonk.tolist()] = mvec[plonk.tolist()]

    # remove "bad data" where we cannot calculate a median value due to already missing values
    if (rem_nomed):
        dvec[np.isnan(mvec)] = np.nan
    return dvec

In [17]:
def filterBlinks_wBlinkData(pupilDataL, pupilDataR, timeInDatetime):
    # filter any blinks and nan values lasting around 250ms (on average)
    # first the single nan occurances are replaced with mean of the values on either sides, 
    # as they are assumed to be from hardware problems
    # for the rest of the blinks, 250ms before and after the nan values are interpolated with a linear function
    # returns a dataframe with pupil size, and timestamp
    # http://faculty.washington.edu/chudler/facts.html
   
    
    # create a dataframe from the pupilsize and time
    pupilData_df = pd.DataFrame(list(zip(timeInDatetime, pupilDataL, pupilDataR)), columns=['timeStamp', 'pupilLeft', 'pupilRight'])
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (22 samples) before and after the blink will also be removed
    extraBlinkSamples = 18
    
    
    #pupilData_woSingleMissingData = pupilData.copy()
    #timeList_woSingleMissingData = timeInDatetime.copy()
    #timeInS_woSingleMissingData = timeInS_Trial[-1]
    
    # in case of single missing data, that are due to hardware error, replace with the mean of the pupil size before and
    # after the nan value
    # missing values will be the same for left and right pupil
    missingVal_Single = np.argwhere(np.isnan(pupilDataL))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list 
    
    
    
    # if no blinks present, return the data
    if len(missingVal_Single) == 0:
        interpolatedNan_bool = np.array([False]*len(pupilData_df['pupilLeft']))
        return pupilData_df, interpolatedNan_bool
    
    # find the index and values to replace for single nan values
    pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataL)-1)) if not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])]
    pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataR)-1)) if not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])]
    
    
    interpolatedNan_bool = np.array([True if ind in dict(pupilData_tuples_replaceSingleNan_left) else False for ind, val in enumerate(pupilDataL)])
    missingData_single = interpolatedNan_bool
    
    # replace the single nan values with the mean of the pupil size on either sides
    indList = -1
    for ind, val in pupilData_tuples_replaceSingleNan_left:
        indList = indList + 1
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilLeft')] = val
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilRight')] = pupilData_tuples_replaceSingleNan_right[indList][1]
        
    
    # again, find the nan values in the pupil size
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index
    # to the next nan value
    
    
    
    # find the nan values again from pupilData['pupilLeft']
    missingVal_Rest_trueFalse = pupilData_df['pupilLeft'].isnull()
    missingVal_Rest = [i for i, x in enumerate(missingVal_Rest_trueFalse) if x]
    
    # if no blinks left, return the current pupilData
    if len(missingVal_Rest) == 0:
        return pupilData_df, interpolatedNan_bool

    
    # in the blinks left, find when the blinks start by finding a difference in the consecutive values of the indices
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    blinkStart_tupleList = [(ind, sum(missingVal_RestDifference[0:ind+1])) for ind, val in enumerate(missingVal_RestDifference) if val != 1]
    
    blinkStart_tupleList_wLength = list()
    
    # create a list of tuples of blink start index and the length of the blink
    ind = -1
    blinkLengthSum = 0
    for blink_ind, blinkStartInd in blinkStart_tupleList:
        ind = ind + 1
        if ind != len(blinkStart_tupleList) - 1:
            
            blinkLength = blinkStart_tupleList[ind+1][0]-blink_ind
            blinkLengthSum = blinkLengthSum + blinkLength
            
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
        else:
            # for the last blink -- all blink lengths summed and subtracted from the length of the list
            # missingVal_RestDifference 
            blinkLength = len(missingVal_RestDifference)-blinkLengthSum
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
     
    
    
    
    # create a vector with True if a blink was at the position
    samplingFrequency = 90
    blinkLengthMax = np.ceil(0.5*samplingFrequency)
    blinkLengthMin = np.ceil(0.075*samplingFrequency)
    blinkStart_tupleList_wLength_allMissingData = [(blinkStart, blinkLength) for blinkStart, blinkLength in blinkStart_tupleList_wLength]    
    blinkStart_tupleList_wLength_maxBlinkLength = [(blinkStart, blinkLength) for blinkStart, blinkLength in blinkStart_tupleList_wLength if blinkLength<blinkLengthMax and blinkLength>blinkLengthMin]
    
    missingData_blinks_bool = np.array([False]*len(pupilData_df['pupilLeft']))
    for blinkStart, blinkLength in blinkStart_tupleList_wLength_maxBlinkLength:
        blinkIndices = np.arange(blinkStart, blinkStart + blinkLength).astype(int)
        if blinkIndices[-1] > len(pupilDataL):
            blinkIndices = np.arange(blinkStart, len(pupilDataL)-1)
        missingData_blinks_bool[blinkIndices] = True
        
    
    
    missingData_blinks_boolAll = np.array([False]*len(pupilData_df['pupilLeft']))
    for blinkStart, blinkLength in blinkStart_tupleList_wLength:
        blinkIndices = np.arange(blinkStart, blinkStart + blinkLength).astype(int)
        if blinkIndices[-1] > len(pupilDataL):
            blinkIndices = np.arange(blinkStart, len(pupilDataL)-1)
        missingData_blinks_boolAll[blinkIndices] = True
        
    missingDataOverall_bool = missingData_single + missingData_blinks_boolAll
    
    # Check for blink bursts: A blink burst (BB) is defined as two or more blinks within 0.5–2.0 s [Horiuchi]
    blinkInd = 0
    blinkBurstList = list()
    
    while blinkInd < len(blinkStart_tupleList_wLength_maxBlinkLength) - 2: # make sure we only go to the second last blink 
        blinkStart = blinkStart_tupleList_wLength_maxBlinkLength[blinkInd][0]
        
        
        blinkBurst = list()
        blinkBurst.append(blinkStart)
        
        for blinkNextInd in range(blinkInd+1, len(blinkStart_tupleList_wLength_maxBlinkLength)-1):
            blinkNextStart = blinkStart_tupleList_wLength_maxBlinkLength[blinkNextInd][0]
            blinkNextEnd = blinkStart_tupleList_wLength_maxBlinkLength[blinkNextInd][1] + blinkNextStart
            
            if blinkNextEnd - blinkStart < 2*90 and blinkNextEnd - blinkStart >= 0.5*90:
                blinkBurst.append(blinkNextStart)
            elif blinkNextStart - blinkStart_tupleList_wLength_maxBlinkLength[blinkNextInd][1] < 0.1*90:
                blinkBurst.append(blinkNextStart)
            else:
                if len(blinkBurst) > 1:
                    blinkBurstList.append(blinkBurst)
                break
        
        blinkInd = blinkNextInd
    
    blinkBurst_timeAndLength = dict()
    # convert the samples from blinkBurstList to time
    
    blinkBurst_timeAndLength['timeStamp'] = [timeInDatetime[blinkBurst[0]] for blinkBurst in blinkBurstList]
    blinkBurst_timeAndLength['nBlinks'] = [len(blinkBurst) for blinkBurst in blinkBurstList]
        
    
    
    # add to vector with start and end of tuple
    #beforeAfterNan = [False]*len(pupilData_df['pupilLeft'])
    #for blinkStart, blinkLength in blinkStart_tupleList_wLength:
    #    beforeAfterNan[blinkStart] = True
    #    beforeAfterNan[blinkStart+blinkLength] = True
    #    #print('start and end points: ', pupilData_df['timeStamp'][blinkStart], pupilData_df['timeStamp'][blinkStart + blinkLength])
    
    
    # create lists with start and end values for the blinks, based on blinkStart_tupleList_wLength, regardless of the blink length
    blink_missingData_startList = [blinkStartInd - extraBlinkSamples if (blinkStartInd - extraBlinkSamples) > 0 else 0 for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    blink_missingData_endList = [blinkStartInd + blinkLength + extraBlinkSamples if (blinkStartInd + blinkLength + extraBlinkSamples) < (len(pupilData_df['pupilLeft'])-1) else (len(pupilData_df['pupilLeft'])-1) for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    # create a list of tuples from the start and end points of the blink
    blink_missingData_startEndTuple = [(blinkStart, blink_missingData_endList[ind]) for ind, blinkStart in enumerate(blink_missingData_startList)] 
    
    
    
        
    
    
    # check if blinks need to be combined - blinksCombine is a list of list of 2 elements, the index of the blinks that should be combined
    blinksCombine = [[ind, ind+1] for ind, blink in enumerate(blink_missingData_startEndTuple[0:-1]) if blink[1] > blink_missingData_startEndTuple[ind+1][0]]
        
    if blinksCombine:
        # combine blinks that need to be combined - if multiple consecutive blinks need to be removed: eg - [1, 2], [2, 3] 
        # are included in the blinksCombine, the combined version should be [1, 3] 
        blinksCombineFinal = list()
        ind = -1
        while ind < len(blinksCombine)-2:
            
            ind = ind + 1
            blinkCombining = blinksCombine[ind]
            blinksCombineFinal.append(blinkCombining)
            while ind < len(blinksCombine)-2 and blinkCombining[1] == blinksCombine[ind+1][0]:
                # change the ending of the last added blink of blinksCombineFinal
                blinksCombineFinal[-1][1] = blinksCombine[ind+1][1]
                ind = ind + 1
            
            
        if len(blinksCombine) == 1:
            blinksCombineFinal = blinksCombine.copy()
            
        
        if blinksCombine[-1][1] != blinksCombineFinal[-1][1]:
            if blinksCombine[-1][0] == blinksCombineFinal[-1][1]:
                blinksCombineFinal[-1][1] = blinksCombine[-1][1]
            else:
                blinksCombineFinal.append(blinksCombine[-1])
        
        
        
        
    #    for w, z in groupby(sorted(list(blinksCombine)), lambda x, y=itertools.count(): next(y)-x):
    #        group = list(z)
    #        blinksCombineFinal.append(tuple((group[0], group[-1])))
        
        for x in sorted(blinksCombineFinal, reverse=True):
            new_start = blink_missingData_startEndTuple[x[0]][0] 
            new_end = blink_missingData_startEndTuple[x[1]][1] 
            
            x_start = x[0]
            x_end = x[1]
            
            # delete also the blinkStart_tupleList_wLength, since it is going to be used to compute other metrics
            for blinkRemove in range(x[1], x[0]-1, -1):
                del blink_missingData_startEndTuple[blinkRemove]
            
            blink_missingData_startEndTuple.insert(x[0], tuple((new_start, new_end)))
    
    
    #blinkAndNonBlinkDurationList = [length/90 for start, length in blinkStart_tupleList_wLength]
    #timeInS_Trial_filter = timeInS_Trial[-1] - sum(blinkAndNonBlinkDurationList) 
    
    
    # remove blinks from data
    for blinkStart, blinkEnd in blink_missingData_startEndTuple:
        pupilData_df.loc[blinkStart:blinkEnd,'pupilLeft'] = np.nan
        pupilData_df.loc[blinkStart:blinkEnd,'pupilRight'] = np.nan
        replaceTrueList = range(blinkStart, blinkEnd+1, 1)
        interpolatedNan_bool[replaceTrueList] = True
    
    
    
    pupilData_df['pupilLeft'] = pupilData_df['pupilLeft'].astype(float).interpolate('linear', limit_direction = 'both')
    pupilData_df['pupilRight'] = pupilData_df['pupilRight'].astype(float).interpolate('linear', limit_direction = 'both')
    
    if pupilData_df.isnull().any().any():
        print('nan values in filtered data')
        #for i,val in enumerate(pupilData_filter[0:5000]):
        #    print(i, val, pupilData_woSingleMissingData[i])
        
    
    return pupilData_df, interpolatedNan_bool, missingData_blinks_bool, missingDataOverall_bool, \
blinkStart_tupleList_wLength_maxBlinkLength, blinkStart_tupleList_wLength_allMissingData, blinkBurst_timeAndLength

In [18]:
def FilterPupilSize_wBlinkData(GazeLog, TimeTyping, subjectAndSessionName):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupil_indWrtValidityL = [1, 2]
    pupil_validityL = 4
    pupilLogL_raw = gazeConvert2ColumnsTo1(GazeLog, pupil_indWrtValidityL, pupil_validityL)
    
    pupil_indWrtValidityR = [1, 2]
    pupil_validityR = 5
    pupilLogR_raw = gazeConvert2ColumnsTo1(GazeLog, pupil_indWrtValidityR, pupil_validityR)
    
    
    # reduce the data to start and end of typing time
    timeTyping_start, timeTyping_startInd = nearestTimePoint(timeGazeLog, TimeTyping['startTime'])
    timeTyping_end, timeTyping_endInd = nearestTimePoint(timeGazeLog, TimeTyping['endTime'])
    
    
    pupilLogL_wDefinedTime = pupilLogL_raw['column1'][timeTyping_startInd:timeTyping_endInd+1]
    pupilLogR_wDefinedTime = pupilLogR_raw['column1'][timeTyping_startInd:timeTyping_endInd+1]
    
    timeGazeLog_wDefinedTime = timeGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    
    timeInS_GazeLog_wDefinedTime = timeInternalGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    timeInS_Difference = [(t - s)/1000000 for s, t in zip(timeInS_GazeLog_wDefinedTime, timeInS_GazeLog_wDefinedTime[1:])]
    timeInS_Difference.insert(0, 0)
    
    
    #timeInS = [sum(timeInS_Difference[:i]) for i, v in enumerate(timeInS_Difference)]
    
    
    #pupilData_df, interpolated_items, missingData_fromBlinks, missingData, missingDataBlink_startNlength, \
    #missingDataAll_startNlength = filterBlinks_wBlinkData(pupilLogL_wDefinedTime, pupilLogR_wDefinedTime, \
    #timeGazeLog_wDefinedTime)
    
    pupilData_df, interpolated_items, missingData_fromBlinks, missingData, missingDataBlink_startNlength, \
    missingDataAll_startNlength, blinkBursts = filterBlinks_wBlinkData(pupilLogL_raw['column1'], pupilLogR_raw['column1']\
                                                                       , timeGazeLog)
    
    
    
    
    #timeGazeLog_plot = np.arange(0, timeInS[-1], 1/90)
    
    #plotPupilSize_checkFilter(pupilData_df, pupilLogL_wDefinedTime, blinkStartAndEnd, 'blink removal', \
    #subjectAndSessionName)
    
    
    
    
    pupilData_df_hampel = dict()
    
    pupilData_df_hampel = pupilData_df.copy()
    pupilData_df_hampel['pupilLeft'] = hampel(pupilData_df['pupilLeft'], 25, 3, False)
    pupilData_df_hampel['pupilRight'] = hampel(pupilData_df['pupilRight'], 25, 3, False)
        
    
        
    return pupilData_df_hampel, interpolated_items, missingData_fromBlinks, missingData, missingDataBlink_startNlength, \
missingDataAll_startNlength, blinkBursts

In [19]:
def EventPartsFromPupilData(EventTimeInKeys, PupilSize_df, full_path):
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    EventTime = dict()    
    EventTime['start'] = list()
    EventTime['end'] = list()
    
    EventIndex = dict()    
    EventIndex['start'] = list()
    EventIndex['end'] = list()
    
    EventBaseline_startKeyTime = list()
    
    for ind, eventStartInKeys in enumerate(EventTimeInKeys['start']):
        
        eventStartTime, eventStartInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), eventStartInKeys)
        eventEndTime, eventEndInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), EventTimeInKeys['end'][ind])
        
        # reading start is the same as trial start
        EventTime['start'].append(eventStartTime)
        EventIndex['start'].append(eventStartInd)
        
        EventTime['end'].append(eventEndTime)
        EventIndex['end'].append(eventEndInd)
        
        
    return EventTime, EventIndex    

In [20]:
def CombineReadingWriting(EventReading, EventWriting, EventReading_index, EventWriting_index):
    
    EventTrial = copy.deepcopy(EventReading)
    EventTrialEnd = [endTime for endTime in EventWriting['end']]
    EventTrial['end'] = EventTrialEnd
    
    EventTrial_index = copy.deepcopy(EventReading_index)
    EventTrialEnd_index = [endTime for endTime in EventWriting_index['end']]
    EventTrial_index['end'] = EventTrialEnd_index
    
    
    return EventTrial, EventTrial_index

In [97]:
def GetBlinkPropertiesFor30sWindow(PupilData_df, MissingData_startNlength, BlinkBursts):
    
    samplingFrequency = 90
    
    
    timeStamp = PupilData_df['timeStamp'].tolist()
    
    
    blinkTimeStart = [timeStamp[indBlink[0]] for indBlink in MissingData_startNlength]
    blinkTimeEnd = [timeStamp[indBlink[0]+indBlink[1]] if indBlink[0]+indBlink[1] < len(timeStamp) else timeStamp[-1] \
                    for indBlink in MissingData_startNlength]
    
    
    
    
    blinkDurationAvg, interBlinkDurationAvg, blinkBurstN = list(), list(), list()
    
    timeStart = timeStamp[0]
    while timeStart < timeStamp[-1]:
        
        timeEnd = timeStart + datetime.timedelta(seconds=30)
        
        trialInd = list()
        
        blinkDurationList, interBlinkDurationList = list(), list()
        
        # blink durations
        for ind, blinkStart in enumerate(blinkTimeStart):
            #print(blinkStart)
            blinkEnd = blinkTimeEnd[ind]
            
            if blinkStart > timeStart:
                #print('add: ', blinkStart)
                    trialInd.append(ind)
            
            if blinkEnd > timeEnd:
                #print('end: ', blinkEnd)
                break
        
        if len(trialInd) > 0:
            if len(trialInd) == 1:
                
                    blinkDurationAvg.append(MissingData_startNlength[ind][1])
                    interBlinkDurationAvg.append(30)
            else:
                blinkDurationList.append([blink[1] for indBlink,blink in enumerate(MissingData_startNlength)\
                                          if indBlink in trialInd])
                interBlinkDurationList = [(blink[0] - MissingData_startNlength[indBlink][0] - \
                                MissingData_startNlength[indBlink][1]) for indBlink, blink in \
                                enumerate(MissingData_startNlength[1:]) if indBlink in trialInd]
                
                blinkDurationAvg.append(np.mean(blinkDurationList)/samplingFrequency)
                interBlinkDurationAvg.append(np.mean(interBlinkDurationList)/samplingFrequency)
        blinkDurationList, interBlinkDurationList = list(), list()
         
    
        trialBurstInd = list()
        
        
        # blink bursts
        for ind, blinkBurstStart in enumerate(BlinkBursts['timeStamp']):
            if blinkBurstStart > timeStart:
                trialBurstInd.append(ind)
                
            if blinkBurstStart - timeStart > datetime.timedelta(seconds=28):
                break
            
        if len(trialBurstInd) > 0:
            if len(trialBurstInd) == 1:
                blinkBurstN.append(1)
            else:
                blinkBurstN.append(len(trialBurstInd))
                
        timeStart = timeEnd
        
        #print(timeStamp[-1], timeStart, timeStamp[-1]-timeStart, datetime.timedelta(seconds=30))
        if timeStamp[-1] - timeStart < datetime.timedelta(seconds=30):
            break
    
    
    Blink_df = pd.DataFrame(list(zip(blinkDurationAvg, interBlinkDurationAvg, blinkBurstN)), columns=['blinkDuration'\
            , 'interBlinkDuration', 'blinkBursts_nBlinks'])
    Blink_df = Blink_df.fillna(0)
    
    
    
    
    return Blink_df

In [98]:
class DataForEverySubject_30sWindow:
    
    blinkBursts = ''
    blinkDuration = ''
    interBlinkDuration = ''
    
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip(self.blinkBursts, self.blinkDuration, self.interBlinkDuration)), columns=[\
                                        'blinkBursts', 'blinkDuration', 'interblinkDuration'])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip(self.blinkBursts, self.blinkDuration, self.interBlinkDuration)), columns=[\
                                        'blinkBursts', 'blinkDuration', 'interblinkDuration'])
        
        #workbook = xlsxwriter.Workbook(self.resultPathName)
        #book = workbook.add_worksheet()
        
        #book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='xlsxwriter')
        #writer.book = book
        #writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        
        dataFrame.to_excel(writer, index = False, header = True)
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        writer.save()
    
    

In [101]:

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4




for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        if 'ac' in root or 'af' in root or 'aq' in root or 'bh1' in root or 'bh2' in root or 'cw' in root or 'kj' in root\
        or 'jm' in root or 'jp' in root:
            continue
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
            
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            print(subjAndSessionName)
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            resultFileName_blink1 = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Subjects\Blinks' + '\\' + subjName \
            + '\\' + subjAndSessionName +  '.xlsx'
            resultFileName_blink2 = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Subjects\BlinkBursts' + '\\' + subjName\
            + '\\' + subjAndSessionName +  '.xlsx'
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root)
            
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = \
            FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, eventTrialsInKeysSelected_reading)
            
            
            if sessionFolderName in dict_noGazeData:
                print('no gaze data present')
                continue
                
            else:
                # filter the data
                pupilData_filtered, interpolatedIndices, missingData_fromBlinks, missingData_overall, \
                missingDataBlink_startNlength, missingDataAll_startNlength, blinkBursts  = FilterPupilSize_wBlinkData(\
                gazeLog, timeTyping, subjAndSessionName)
                
                """
                print('missing data ratio: ', sum(missingData_overall)/len(missingData_overall))
                print('interpolated data ratio: ', sum(interpolatedIndices)/len(interpolatedIndices))
                print('missing data from blinks ratio: ', sum(missingData_fromBlinks)/len(missingData_fromBlinks))
                
                blinkDurations = [item[1] for item in missingDataBlink_startNlength]
                fig=plt.figure(), plt.hist(blinkDurations)
                plt.title('blinkDurationHistograms')
                pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Blinks\blinkDurationHistograms'+'\\'+subjName+'\\'+subjAndSessionName, 'wb'))
                
                missingDataAllDurations = [item[1] for item in missingDataAll_startNlength]
                fig=plt.figure(), plt.hist(missingDataAllDurations)
                plt.title('MissingDataLengthHistograms')
                pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Blinks\MissingDataLengthHistograms'+'\\'+subjName+'\\'+subjAndSessionName, 'wb'))
                
                
                """
                #print(missingDataBlink_startNlength)
                
                # divide trials into reading and writing
                eventReading, eventReading_index = EventPartsFromPupilData(eventTrialsInKeysSelected_reading, pupilData_filtered, root)
                eventWriting, eventWriting_index = EventPartsFromPupilData(eventTrialsInKeysSelected_writing, pupilData_filtered, root)
                
                eventTrial, eventTrial_index = CombineReadingWriting(eventReading, eventWriting, eventReading_index, eventWriting_index)
                
                #print('eventReadingIndex: ', eventReading_index)
                #print('')
                #print('eventTrialIndex:  ', eventTrial_index)
                
                # compute blink durations for all trials
                blink = GetBlinkPropertiesFor30sWindow(pupilData_filtered, missingDataBlink_startNlength, blinkBursts)
                
                nBlinks_blinkBursts = blink['blinkBursts_nBlinks'].tolist()
                blinkDuration = blink['blinkDuration'].tolist()
                interBlinkDuration = blink['interBlinkDuration'].tolist()
                
                
                
            if '1stPart' in root:
                print('1stPart')
                blinkDuration1 = blinkDuration
                interBlinkDuration1 = interBlinkDuration
                
                nBlinks_blinkBursts1 = nBlinks_blinkBursts
                continue
            
            if '2ndPart' in root:
                print('2ndPart')
                blinkDuration2 = blinkDuration
                interBlinkDuration2 = interBlinkDuration
                
                nBlinks_blinkBursts2 = nBlinks_blinkBursts
                
                blinkDuration = blinkDuration1 + blinkDuration2 #ct #result of this and 
                nBlinks_blinkBursts1, blinkDuration1, interBlinkDuration1 = list(), list(), list()
                
                
            # save the blink data
            dataToSave = DataForEverySubject_30sWindow()
            dataToSave.blinkBursts = nBlinks_blinkBursts
            dataToSave.blinkDuration = blinkDuration
            dataToSave.interBlinkDuration = interBlinkDuration
            dataToSave.resultPathName = resultFileName_blink1
            
            print(dataToSave.printInfo())
            
            
            
            
            dataToSave.AddToFile()
            

subject path E:\Data\Data\le\1\2019-02-18-13-20-38_1
le__1__2019-02-18-13-20-38_1
subject and session name:  le__1__2019-02-18-13-20-38_1
    blinkBursts  blinkDuration  interblinkDuration
0             1      12.000000           30.000000
1             1      12.000000           30.000000
2             1       0.136111            9.755556
3             3       0.201852            6.248148
4             1       0.129630           32.788889
5             1      14.000000           30.000000
6             1       0.138889           23.838889
7             1       0.150000           11.627778
8             2       0.140000           15.377778
9             1      10.000000           30.000000
10            2       0.122222           13.063889
11            1       8.000000           30.000000
12            2       0.144444           12.716667
13            1      23.000000           30.000000
14            2       0.248148            8.585185
15            1       0.133333           33.15

subject path E:\Data\Data\le\3\2019-02-21-15-01-4_1stPart_1
le__3__2019-02-21-15-01-4_1stPart_1
subject and session name:  le__3__2019-02-21-15-01-4_1stPart_1
1stPart
subject path E:\Data\Data\le\3\2019-02-21-15-25-56_2ndPart_1
le__3__2019-02-21-15-25-56_2ndPart_1
subject and session name:  le__3__2019-02-21-15-25-56_2ndPart_1
2ndPart
    blinkBursts  blinkDuration  interblinkDuration
0             2       0.179167           10.484444
1             2       0.133333            2.795556
2             2       0.133333           17.402778
3             1       0.142593           30.000000
4             1       0.137037           11.944444
5             2       0.150000           48.466667
6             1       0.114815           30.000000
7             1       0.153333           30.000000
8             1       0.142222           30.000000
9             1       0.150327           55.405556
10            1       0.218519           44.300000
11            1       0.234722           30.000000


subject path E:\Data\Data\le\5_MS\2019-03-04-12-00-15_2
le__5_MS__2019-03-04-12-00-15_2
subject and session name:  le__5_MS__2019-03-04-12-00-15_2
    blinkBursts  blinkDuration  interblinkDuration
0             1       0.088889           25.711111
1             1       0.088889           23.533333
2             1       0.144444           23.450000
3             1       0.166667           33.533333
4             1       0.116667           21.283333
5             2       0.137037            2.166667
6             4       0.148485            4.090909
7             2       0.203704            7.562963
8             1       0.177778           11.607407
9             1       0.133333            7.813333
10            3       0.200000            5.893333
11            1       0.144444            7.413889
12            4       0.124242            6.454545
13            1       0.133333           37.116667
14            1       0.138889           18.133333
15            1       0.107407       

subject path E:\Data\Data\ls1\3\2019-02-13-15-58-26_1
ls1__3__2019-02-13-15-58-26_1
subject and session name:  ls1__3__2019-02-13-15-58-26_1
    blinkBursts  blinkDuration  interblinkDuration
0             7       0.130952            0.957540
1             9       0.159804            0.714706
2             8       0.217921            2.033692
3             1       9.000000           30.000000
4             3       0.169630            2.001481
5             3       0.148611            3.763889
6             6       0.130159            1.897884
7             2       0.136111            2.598611
8             8       0.162698            0.911905
9             3       0.120741            3.157778
10            1       0.197222            4.802778
11            7       0.141546            1.164251
12            8       0.153333            1.004444
13            5       0.178519            1.326667
14            1       0.100000           22.796296
15            1       8.000000           30

subject path E:\Data\Data\ls1\5\2019-02-21-17-11-42_1
ls1__5__2019-02-21-17-11-42_1
subject and session name:  ls1__5__2019-02-21-17-11-42_1
    blinkBursts  blinkDuration  interblinkDuration
0             2       0.155556            5.676190
1             4       0.137607            5.724786
2             1      10.000000           30.000000
3             1       0.125926           20.855556
4             1      15.000000           30.000000
5             6       0.150370            1.294074
6             4       0.125641            2.287179
7             2       0.112698            4.255556
8             1       0.161111            7.502778
9             4       0.132222            5.142222
10            2       0.135185            3.455556
11            2       0.165079            4.566667
12            3       0.195062            2.496296
13            5       0.161806            1.782639
14            5       0.171111            2.479259
15            5       0.204274            1

...

subject path E:\Data\Data\rh\1\2019-03-07-16-17-30_1
rh__1__2019-03-07-16-17-30_1
subject and session name:  rh__1__2019-03-07-16-17-30_1
    blinkBursts  blinkDuration  interblinkDuration
0             2       0.137037            6.262963
1             2       0.152222            2.892222
2             2       0.155556            6.961111
3             1       0.108333            5.375000
4             3       0.106667            3.545556
5             2       0.112346            3.481481
6             1       0.113889            7.333333
7             2       0.118056            7.113889
8             1       0.114815            4.992593
9             3       0.111111            4.266667
10            1       0.094444            7.422222
11            1       0.119444            8.213889
12            2       0.143056            5.254167
13            2       0.137037            9.137037
14            1       0.119444            7.227778
15            4       0.110101            2.80

subject path E:\Data\Data\rh\2\2019-03-13-13-40-7_2
rh__2__2019-03-13-13-40-7_2
subject and session name:  rh__2__2019-03-13-13-40-7_2
    blinkBursts  blinkDuration  interblinkDuration
0             3       0.101587            7.301587
1             2       0.107407            3.014815
2             1       0.139683            4.277778
3             2       0.120370            8.798148
4             3       0.112963            1.353704
5             3       0.109259            5.305556
6             2       0.115556            4.745556
7             2       0.117778            4.180000
8             1       0.094444            4.709722
9             5       0.114530            1.844444
10            2       0.116667            5.598148
11            4       0.117284            3.264198
12            4       0.102778            8.776389
13            1       0.094444           55.222222
14            1       9.000000           30.000000
15            1       9.000000           30.00000

subject path E:\Data\Data\rh\4\2019-04-11-13-40-31_2
rh__4__2019-04-11-13-40-31_2
subject and session name:  rh__4__2019-04-11-13-40-31_2
    blinkBursts  blinkDuration  interblinkDuration
0             4       0.127778            2.689815
1             4       0.111111            1.308466
2             5       0.112346            1.962346
3             2       0.111111            2.676667
4             3       0.104938            3.239506
5             4       0.111111            2.759829
6             4       0.104444            2.855556
7             5       0.142361            1.751389
8             7       0.111877            0.977395
9             6       0.107407            2.209630
10            5       0.117361            1.775694
11            3       0.112346            3.245679
12            6       0.114141            1.400505
13            4       0.105128            2.502564
14            4       0.100000            3.005556
15            5       0.107870            1.30

subject path E:\Data\Data\ys\1\2019-01-14-15-25-55_2
ys__1__2019-01-14-15-25-55_2
subject and session name:  ys__1__2019-01-14-15-25-55_2
    blinkBursts  blinkDuration  interblinkDuration
0             1       0.185185           15.496296
1             2       0.198148            2.151852
2             4       0.203030            2.586869
3             3       0.179630           27.657407
4             1       8.000000           30.000000
5             1       8.000000           30.000000
6             1       8.000000           30.000000
7             1       8.000000           30.000000
8             1       0.100000           13.477778
9             2       0.130556            7.869444
10            1       0.124444           23.400000
11            1       9.000000           30.000000
12            1       9.000000           30.000000
13            1       0.116667           31.455556
14            1       0.122222           20.972222
15            1      10.000000           30.00

subject path E:\Data\Data\ys\4_MS\2019-01-30-11-57-3_2
ys__4_MS__2019-01-30-11-57-3_2
subject and session name:  ys__4_MS__2019-01-30-11-57-3_2
no gaze data present
subject path E:\Data\Data\ys\5\2019-01-31-13-13-2_1
ys__5__2019-01-31-13-13-2_1
subject and session name:  ys__5__2019-01-31-13-13-2_1
    blinkBursts  blinkDuration  interblinkDuration
0             5       0.206173            3.244444
1             2       0.109259            6.272222
2             5       0.150794            3.871429
3             1       0.162963            7.948148
4             1       0.166667            7.311111
5             2       0.129630            8.314815
6             1       0.103704           15.748148
7             2       0.100000            1.640741
8             6       0.113072            1.725490
9             1       0.115741            3.018519
10            2       0.112698            3.982540
11            3       0.132323            3.231313
12            3       0.231111       